In [1]:
import numpy as np
import soundfile as sf
import onnxruntime as rt
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Processor

sess_options = rt.SessionOptions()
sess_options.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_ALL

rt.get_available_providers()

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

SystemError: <built-in function __import__> returned a result with an error set

['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']

In [3]:
processor = Wav2Vec2FeatureExtractor.from_pretrained("models/29epoch combined with egg 8051 real865")
model = rt.InferenceSession('model.onnx', sess_options=sess_options, providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])

In [14]:
id2label = {
    0: "angry",
    1: "fear",
    2: "happy",
    3: "neutral",
    4: "sad"}

def predict(filepath):
    speech_array, sr = sf.read(filepath)
    features = processor(speech_array, sampling_rate=16000, return_tensors="np")
    input_values = features.input_values
    onnx_outputs = model.run(None, {model.get_inputs()[0].name: input_values})[0]
    prediction = np.argmax(onnx_outputs, axis=-1)
    return prediction.squeeze().tolist()


id2label[predict('data/TIL_train_normalized/angry/00f5ccb1da.wav')]

'angry'